Carregando os dados

In [ ]:
import pandas as pd
!pip install dash
!pip install plotly
!pip install pyvis
!pip install python-louvain
users_df = pd.read_csv("/content/bx_teste.csv", encoding="iso-8859-1")
users_df

Verificando as comundades

In [80]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community
from tqdm import tqdm

# Criando um grafo direcionado
G = nx.DiGraph()

# Convertendo todos os IDs para strings antes de adicionar ao grafo
users_df["User_ID"] = users_df["User_ID"].astype(str)
users_df["ISBN"] = users_df["ISBN"].astype(str)

# Verificando duplicatas e limpando os dados
users_df.drop_duplicates(subset=["User_ID", "ISBN"], inplace=True)

# Adicionando nós (usuários e ISBN)
unique_nodes = set(users_df["User_ID"]) | set(users_df["ISBN"])

# Adicionando nós ao grafo
for node in tqdm(unique_nodes, desc="Adicionando nós"):
    G.add_node(str(node))

# Adicionando arestas (interações entre usuários e livros)
for _, row in tqdm(users_df.iterrows(), total=len(users_df), desc="Adicionando arestas"):
    G.add_edge(row["User_ID"], row["ISBN"])

# Detecção de comunidades usando Louvain
louvain_communities = list(tqdm(community.greedy_modularity_communities(G), desc="Detectando comunidades (Louvain)"))

# Imprimindo os resultados
print("Resultados da detecção de comunidades:\n")
print("Método Louvain:")
print(f"Número de comunidades: {len(louvain_communities)}")

Detectando comunidades (Louvain): 100%|██████████| 2790/2790 [00:00<00:00, 1070347.40it/s]


Resultados da detecção de comunidades:

Método Louvain:
Número de comunidades: 2790


Calculando a centralidade

In [ ]:
# Calculando a centralidade de intermediação para todos os nós
node_betweenness = betweenness_centrality(G)

Sugerindo os livros pelo ISBN

In [81]:
# Sugestão de livros baseada em centralidade de intermediação
top_books = sorted((node for node in node_betweenness if node in users_df["ISBN"].unique()), key=lambda x: node_betweenness[x], reverse=True)[:5]
print("\nTop 5 livros recomendados com base na centralidade:")
for book in top_books:
    print(f"Livro: {book}, Centralidade: {node_betweenness[book]}")


Top 5 livros recomendados com base na centralidade:
Livro: 1931643210, Centralidade: 0.0
Livro: 1558007075, Centralidade: 0.0
Livro: 068985790X, Centralidade: 0.0
Livro: 3518393707, Centralidade: 0.0
Livro: 8495359391, Centralidade: 0.0


Gerando Grafico teste 1

In [ ]:
import matplotlib.colors as mcolors

# Melhorando o layout
pos = nx.kamada_kawai_layout(G)  # Este layout pode oferecer uma visualização melhor para grafos complexos

# Ajustando a paleta de cores para diferenciação clara
colors = list(mcolors.TABLEAU_COLORS.keys())  # Usando cores do Tableau para melhor visualização
community_colors = {node: colors[i % len(colors)] for i, community in enumerate(louvain_communities) for node in community}

# Ajustando tamanho dos nós e arestas
node_sizes = [node_betweenness[node] * 1000 for node in G.nodes()]  # Aumenta o tamanho dos nós baseado na centralidade
edge_widths = [0.5 for _ in G.edges()]  # Espessura uniforme para todas as arestas, ajuste conforme necessário

plt.figure(figsize=(12, 10))
nx.draw(G, pos,
        node_color=[community_colors[node] for node in G.nodes()],
        node_size=node_sizes,
        width=edge_widths,
        with_labels=False,  # Mudar para True se quiser ver todos os rótulos
        edge_color='lightgray')  # Arestas menos destacadas para melhor visualização dos nós

# Adicionando uma legenda para as comunidades
for i, community in enumerate(louvain_communities):
    plt.scatter([], [], c=colors[i % len(colors)], label=f'Comunidade {i+1}')
plt.legend(title="Comunidades", loc='upper left', bbox_to_anchor=(1, 1))

plt.title('Visualização Melhorada das Comunidades Louvain')
plt.show()


Gerando Grafico teste 2

In [ ]:
import plotly.graph_objects as go
import networkx as nx
from networkx.algorithms import community
from matplotlib.colors import to_hex

# Supondo que G e louvain_communities já estejam definidos e calculados.

# Preparar posições dos nós com um layout escolhido
pos = nx.kamada_kawai_layout(G)

# Convertendo cores do Matplotlib para Hexadecimal
colors = [to_hex(c) for c in mcolors.TABLEAU_COLORS.values()]
community_color = {node: colors[i % len(colors)] for i, community in enumerate(louvain_communities) for node in community}

# Preparar dados dos nós
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    text=[f'Node: {node}' for node in G.nodes()],
    mode='markers+text',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale='Viridis',
        size=10,
        color=[community_color[node] for node in G.nodes()],  # Cores em formato hexadecimal
        line_width=2))

# Preparar dados das arestas
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Criar a figura
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Network graph made with Python',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        text="Python code: NetworkX and Plotly",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002)],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()


Gerando Grafico teste 3

In [ ]:
import dash
from dash import html, dcc, Input, Output
import plotly.graph_objects as go
import networkx as nx
from networkx.algorithms import community
from matplotlib.colors import to_hex

# Suponha que 'G' e 'louvain_communities' estejam definidos e calculados.
colors = [to_hex(c) for c in mcolors.TABLEAU_COLORS.values()]  # Cores em hexadecimal

# Posições dos nós com layout Kamada-Kawai
pos = nx.kamada_kawai_layout(G)

app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Dropdown para seleção de comunidades
community_options = [{'label': f'Comunidade {i + 1}', 'value': i} for i in range(len(louvain_communities))]
community_dropdown = dcc.Dropdown(
    options=community_options,
    id='community-dropdown',
    style={'width': '50%', 'padding': '10px'}
)

# Componente Graph de Plotly
graph_component = dcc.Graph(id='network-graph')

app.layout = html.Div([
    html.Div([community_dropdown], style={'display': 'flex', 'justifyContent': 'center'}),
    graph_component
], style={'fontFamily': 'Arial, sans-serif'})

@app.callback(
    Output('network-graph', 'figure'),
    Input('community-dropdown', 'value')
)
def update_graph(selected_community_index):
    fig = go.Figure()

    # Arestas em cinza para menos destaque
    for edge in G.edges():
        fig.add_trace(go.Scatter(
            x=[pos[edge[0]][0], pos[edge[1]][0]], y=[pos[edge[0]][1], pos[edge[1]][1]],
            line=dict(color='gray', width=1),
            mode='lines',
            hoverinfo='none'
        ))

    # Nós destacados pela seleção de comunidade
    for i, community in enumerate(louvain_communities):
        node_x = [pos[node][0] for node in community]
        node_y = [pos[node][1] for node in community]
        node_color = colors[i % len(colors)] if i == selected_community_index else '#cccccc'

        fig.add_trace(go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            marker=dict(size=10, color=node_color),
            text=[node if i == selected_community_index else '' for node in community],  # Mostra rótulos somente para a comunidade selecionada
            hoverinfo='text',
            hovertext=[f'Node: {node}' for node in community]
        ))

    fig.update_layout(
        showlegend=False,
        hovermode='closest',
        margin={'b': 20, 'l': 5, 'r': 5, 't': 40},
        xaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
        yaxis={'showgrid': False, 'zeroline': False, 'showticklabels': False},
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
